In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Select a test signal
import librosa
filename = librosa.util.example_audio_file()

# Load the signal

In [ ]:
from utils import load_signal
y, sr = load_signal(filename)
# Apply some light preprocessing
from utils import preprocess_signal
y = preprocess_signal(y)
# Plot the signal in the time domain
n = 1024*256
t = np.arange(n)/sr*1000
plt.plot(t, y[:n])
plt.xlabel('Time [ms]');

In [ ]:
from IPython.display import display, Audio
display(Audio(y, rate=sr))

# Define the main parameters
* The number of frequency channel for the stft `stft_channels`
* The hop_size in time `hop_size`

In [ ]:
# Here we use the default paramters. We recommend using them.
from hparams import HParams
stft_channels = HParams.stft_channels # 1024
hop_size = HParams.hop_size # 256


# Build the time frequency system
The system will be used for the following operation:
* Compute the STFT
* Compute the spectrogram
* Invert the STFT
* Reconstruc the phase and invert the spectrogram

In [ ]:
from stft import GaussTF
stft_system = GaussTF(hop_size=hop_size, stft_channels=stft_channels)

# Compute the spectrogram

In [ ]:
Y = stft_system.spectrogram(y)

In [ ]:
# We use the log spectrogram as we have a logarithmic perception of sound energy.
from transforms import log_spectrogram
log_Y= log_spectrogram(Y)

In [ ]:
plt.figure(dpi=200, figsize=(10,3))
plt.imshow(log_Y[:,:n//hop_size])
plt.colorbar()

# Invert the representation

In [ ]:
from transforms import inv_log_spectrogram
from metrics import projection_loss

new_Y = inv_log_spectrogram(log_Y)
print(projection_loss(new_Y, Y))

In [ ]:
new_y = stft_system.invert_spectrogram(new_Y)


In [ ]:
from metrics import projection_loss
new_Yp = stft_system.spectrogram(new_y)
print(projection_loss(new_Y, new_Yp))

In [ ]:
display(Audio(new_y, rate=sr))